In [1]:
# knn, SVM, random forest
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas
print("starting")
df = pandas.read_csv("tweets.csv", sep='\t', error_bad_lines=False) #ignore bad lines
print(df.columns)
print(df.shape)


starting
Index([u'URL', u'Keywords', u'Keyword Count', u'DateTime', u'Favorite Count',
       u'Retweet', u'Lang', u'LinkCount', u'Link1', u'Link2', u'Link3',
       u'Author', u'Text', u'Followers', u'Friends', u'Location', u'Timezone',
       u'UTC Offset'],
      dtype='object')
(2414, 18)


In [82]:
# set threshold for binary classifier
#retweets = df.Retweet
threshold = 5000
# train_labels = []
# for num in retweets:
#     if num >= threshold:
#         train_labels.append(1) #viral
#     else:
#         train_labels.append(0) #not viral
df['Viral'] =df['Retweet'].apply(lambda retweet: 1 if retweet >= threshold else 0)
train_labels = df.Viral.values
labels = list(set(train_labels))




In [103]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem.snowball import *
stemmer = SnowballStemmer('english')
#stemmer = PorterStemmer('english')

#stopwords?
stop = ['amp', 'cc', 'did', 'don', 'rt', 'll', 'oh', 've', 'yes', 'let', 'going', 'via', 're', 'tweet' ]
#stop = []
#http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
# preProcess(str):
#     url_pattern = re.compile(r'http(s?)://[\w./]+')
#     pic_pattern = re.compile(r'pic.twitter.com/[\w.]+')
#     str = pic_pattern.sub("", str)
#     str = url_pattern.sub("", str)
#     return str
# http://shahmirj.com/blog/extracting-twitter-usertags-using-regex
class NoUrls_CountVectorizer(CountVectorizer):
    def build_preprocessor(self):
        url_pattern = re.compile(r'http(s?)://[\w./]+')
        pic_pattern = re.compile(r'pic.twitter.com/[\w.]+')
        preprocessor = super(NoUrls_CountVectorizer, self).build_preprocessor()
        return lambda doc: (pic_pattern.sub('', url_pattern.sub('', preprocessor(doc)) ))

class NoUrls_Stemmed_CountVectorizer(CountVectorizer):
    def build_preprocessor(self):
        url_pattern = re.compile(r'(?:\@|https?://)\S+')
        #url_pattern = re.compile(r'http(s?)://)[\w./]+')
        pic_pattern = re.compile(r'pic.twitter.com/[\w.]+')
        at_pattern = re.compile(r' (?<=^|(?<=[^a-zA-Z0-9-_\\.]))@([A-Za-z]+[A-Za-z0-9_]+)')
        preprocessor = super(NoUrls_Stemmed_CountVectorizer, self).build_preprocessor()
        return lambda doc: (pic_pattern.sub('', url_pattern.sub('', preprocessor(doc)) ))
   #right now just doing splits on whitespace and stemming 
    def build_tokenizer(self):
        tokenizer = super(NoUrls_Stemmed_CountVectorizer, self).build_tokenizer()
        return lambda doc: (stemmer.stem(w) for w in tokenizer(doc))

#bigrams? ngram_range=(1,2)
vectorizer = NoUrls_Stemmed_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')
   
#example processing
# tweet = "RT @femaIes: When you gotta go visit your broads in Atlanta https://t.co/pCau23tv3q"
# tweet = unicode(tweet, "ascii", "ignore")
# print tweet
# print 'Preprocess:', vectorizer.build_preprocessor()(tweet)
# print
# print 'Analyze:', vectorizer.build_analyzer()(tweet)
# vectorizer.fit_transform([tweet])
# vectorizer.vocabulary_  

print "done"

done


In [100]:
documents = df.Text
X_count = vectorizer.fit_transform(documents)
print X_count.toarray()
print vectorizer.get_feature_names()


[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[u'00', u'000', u'00in', u'00mst', u'00pm', u'01', u'02', u'02in', u'03', u'04', u'05pm', u'06', u'07', u'08', u'081', u'09', u'0c', u'0f', u'0km', u'0kts', u'0mm', u'0mph', u'10', u'100', u'1000', u'1002', u'1004', u'1005', u'1006', u'1007', u'1008', u'1009', u'100th', u'100x', u'101', u'1011', u'1014', u'102', u'1026', u'1039', u'1080p', u'10c', u'11', u'111', u'117', u'11am', u'12', u'120', u'122', u'125', u'13', u'132', u'134', u'138', u'14', u'142', u'14th', u'15', u'150', u'1500', u'15min', u'16', u'160407', u'160409', u'160411', u'160412birthday', u'160413', u'16gb', u'16u', u'17', u'175', u'176', u'18', u'186', u'18th', u'19', u'1911', u'1983', u'1988', u'1991', u'1992', u'1999', u'1am', u'1d', u'1km', u'1m', u'1million', u'1oc', u'1st', u'1tb', u'20', u'2000', u'2004', u'2009', u'2011', u'2012', u'2013', u'2014', u'2016', u'2017', u'2020', u'2025', u'

In [49]:
#http://scikit-learn.org/stable/auto_examples/feature_stacker.html
#trying to append to feature list X
#http://stackoverflow.com/questions/19466868/how-do-i-do-classification-using-tfidfvectorizer-plus-metadata-in-practice
import scipy
linkCount = [df.LinkCount]
linkCount = np.array(linkCount)
X_metadata = scipy.sparse.csr_matrix(linkCount.T) #must transpose to get shape (___, 1)
#print X_metadata
#print X_count
#X_metadata = [df.LinkCount.as_matrix()]
print X_count.shape
print X_metadata.shape
X = scipy.sparse.hstack([X_count, X_metadata])
print X.toarray()

print X.shape

(2414, 8035)
(2414, 1)
[[0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
(2414, 8036)


In [51]:
train_labels = np.array(train_labels)
train_features = X

print "train labels: "
print train_labels
print labels
print 
print "train features:"
print train_features.toarray()

train labels: 
[1 1 0 ..., 0 0 0]
[0, 1]

train features:
[[0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [53]:
#Make SVM classifier
from sklearn import svm
#linear or not?
name = "Liblinear"
#loss=?, penalty=?, dual=False?, tol=1e-3
classifier = svm.LinearSVC()
classifier.fit(train_features, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [85]:
#Make a pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',   NoUrls_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')),
    ('classifier',  svm.LinearSVC()) ])

from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
#adapted from http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
k_fold = KFold(n=len(df), n_folds=10)
scores = []
confusion = np.array([[0, 0], [0, 0]])

for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['Text'].values
    train_y = df.iloc[train_indices]['Viral'].values
    
    test_text = df.iloc[test_indices]['Text'].values
    test_y = df.iloc[test_indices]['Viral'].values
    
    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #update totals
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)
print('Total tweets classified:', len(df))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

('Total tweets classified:', 2414)
('Score:', 0.11922603712077398)
Confusion matrix:
[[2224  748]
 [ 863  748]]


In [94]:
#Make kNN classifier
#NearestCentroid, KNeighborsClassifier
from sklearn import neighbors
name = "kNN"
# classifier = neighbors.
# KNeighborsClassifier(n_neighbors = 10)
# classifier.fit(train_features, train_labels)

#Make a pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',   NoUrls_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')),
    ('classifier',  neighbors.KNeighborsClassifier(n_neighbors = 4)) ])

from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
#adapted from http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
k_fold = KFold(n=len(df), n_folds=10)
scores = []
confusion = np.array([[0, 0], [0, 0]])

for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['Text'].values
    train_y = df.iloc[train_indices]['Viral'].values
    
    test_text = df.iloc[test_indices]['Text'].values
    test_y = df.iloc[test_indices]['Viral'].values
    
    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #update totals
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)
print('Total tweets classified:', len(df))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

('Total tweets classified:', 2414)
('Score:', 0.13888145670754368)
Confusion matrix:
[[2235  737]
 [ 872  739]]


In [84]:
#Make Random Forest clasisifier
from sklearn import ensemble
# name = "randomforest"
# classifier = ensemble.RandomForestClassifier(n_estimators=100)
# classifier.fit(train_features, train_labels)

#Make a pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',   NoUrls_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')),
    ('classifier',  ensemble.RandomForestClassifier(n_estimators=100)) ])

from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
#adapted from http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
k_fold = KFold(n=len(df), n_folds=10)
scores = []
confusion = np.array([[0, 0], [0, 0]])

for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['Text'].values
    train_y = df.iloc[train_indices]['Viral'].values
    
    test_text = df.iloc[test_indices]['Text'].values
    test_y = df.iloc[test_indices]['Viral'].values
    
    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #update totals
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)
print('Total tweets classified:', len(df))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

C:\Users\Leandra\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


('Total tweets classified:', 2414)
('Score:', 0.09441786283891547)
Confusion matrix:
[[2247  966]
 [1113  980]]
